# Ant Colony Optimisation (ACO)

ACO is a simple metaheuristics that only requires 2 mandatory parameters in its most simple implementation:
- Number of ants
- Pheromone Evaporation


The steps are the following (here for a TSP but works in general):
1. Initialise pheromone and problem setup
2. Loop until termination criterion is reached:<br>
    a) Construct a feasible solution for each ant using the pheromone matrix<br>
    b) Evaluate quality of all ant solution<br>
    c) Update pheromone (evaporation and deposition)



## Import necessary libraries

In [ ]:
"""
Created on Mon Okt 28 2013

@author:    S.H. Hesse
            For the use in my Ph.d Research
         
@purpose:   Ant Colony Optimization method for TSP.
            Based on research by M. Dorigo
            
Adapted for use in Jupyter Lab with additional interactive plotting by A. Kaps
"""

import numpy as np
import matplotlib.pyplot as plt
import scipy.spatial.distance as ssd

##  Auxiliary graph reading function
Reads the vertices of a fully connected graph from a given file

In [ ]:
def readFile(filename):
    Graph = []
    in_file = open(filename, 'r')
    for line in in_file:
        tmp = line.split(',')
        row = []
        for col in range(len(tmp)):
            row.append(float(tmp[col].strip()))
        Graph.append(row)
    return np.array(Graph)

## Auxiliary plotting function
Given the nodes of a fully-connected graph as well as an axes handle this function plots the graph

In [ ]:
def plot_graph(ax,x,y):
    ax.set_aspect('equal')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    # Plot cities
    ax.scatter(x, y, c='grey', s=100, zorder=2)
        
    # Here we are plotting all the links between cities
    for i in range(len(x)):  
        for j in range(len(y)):
            ax.plot([x[i],x[j]],[y[i],y[j]], c='grey', ls='-', alpha=0.1, zorder=1)

## Single solution construction and pheromone updating

In [ ]:
# Selects the subsequent city in the path of an ant
# given probablilities determined below
def selectCity(probs):
    w_cumsum = np.cumsum(probs)
    choice_l = np.random.rand()
    for i in np.arange(0,len(w_cumsum)):
        if choice_l <= w_cumsum[i]:
            choice = i
            break    
    return choice


# Constructs a solution for a single ant
def constructSolution(locPher, dist, N, alpha=1, beta=5):
    # Initialise  variables by randomly choosing starting city
    sol = np.zeros((N,2),dtype="int")
    startCity = np.random.randint(0,N)
    city = startCity
    Nsol = np.arange(N)
    Nsol = Nsol[Nsol!=city]
    
    cnt = 0
    # Loop through all remaining cities
    while len(Nsol):
        # This scheme was presented in lecture as "advanced TSP scheme"
        posPher = locPher[city,Nsol]**alpha
        posDist = dist[city,Nsol]**(-beta)
        pick = []
        for j in Nsol:
            pick.append( ( locPher[city,j]**alpha * dist[city,j]**-beta ) / ( np.dot(posPher,posDist.T) ) )

        # Select the next city
        oldCity = city
        city = Nsol[selectCity(pick)]
    
        # Update the solution and continue
        sol[cnt] = [oldCity, city]
        Nsol = Nsol[Nsol!=city]
        cnt += 1

    sol[cnt] = [city, startCity]
    return sol.tolist()

# Compute delta pheromone for advanced scheme
def updatePheromone(locSol,locF_eval, N, Q = 1):
    deltaPher = np.zeros((N,N))
    for edge in locSol:
        deltaPher[edge[0],edge[1]] = Q / locF_eval
        deltaPher[edge[1],edge[0]] = Q / locF_eval

    return deltaPher

probs= np.ones(4) * 0.25
selectCity(probs)

## Main optimisation loop

In [ ]:
def ACO(graph_file, k, gen_max, rho = 0.5, plotting = True):
    # Read the graph and plot it 
    graph = readFile(graph_file)
           
    # Now initialise some variables such as size, pheromone and distance matrix
    N = len(graph)
        
    nrPher = np.sum(range(1,N))
    pher = np.ones((N,N))*0.0001
    
    dist = ssd.squareform(ssd.pdist(graph))
    cnt = 0
    
    # Plot initial graph and prepare interactive plotting
    if plotting:
        plt.ion()
        fig = plt.figure(1)
        ax = fig.add_subplot(111)
        x = graph[:,0]
        y = graph[:,1]
        plot_graph(ax,x,y)
        fig.canvas.draw()
        plt.pause(0.5)
        fig = plt.figure(2)
        ax = fig.add_subplot(111)
        
    # Start the optimisation loop itself
    while True:
        cnt += 1

        sol = []
        F_eval = []
        # Do for each ant
        for ant in range(k):
            # Construct a solution and add it to the list of all solutions
            currSol = constructSolution(pher, dist, N)
            sol.append(currSol)

            # Evaluate the quality of the solution
            f_eval = 0
            for i in range(N):
                f_eval = f_eval + dist[currSol[i][0]][currSol[i][1]]
            F_eval.append(f_eval)
        
        # Plot the current best solution
        if plotting:
            ax.clear()
            plot_graph(ax,x,y)
            
            path = sol[np.argmin(F_eval)]
            for line in path:
                ax.plot([x[line[0]], x[line[1]]], [y[line[0]], y[line[1]]], c='orange', ls='-', alpha=1.0, linewidth=3, zorder=1)
            ax.set_title('Iteration: {}\nCurrent best solution'.format(cnt))
            fig.canvas.draw()
            plt.pause(0.1)
    
        # Update pheromone values (again advanced scheme)
        pher *= (1-rho)
        for ant in range(k):
            deltaPher = updatePheromone(sol[ant],F_eval[ant], N)
            pher += deltaPher
    
        if cnt == gen_max:
            break
    
    if plotting:
        pher[pher<1e-3] = 0
        print("="*40)
        print("Finished ACO after {} generations!".format(gen_max))
        print("Shortest path has length {:f}.".format(min(F_eval)))
        print("Path was used by {} of the {} ants!".format(np.count_nonzero(F_eval == min(F_eval)), k))
        fig = plt.figure(3)
        ax = fig.add_subplot(111)
        plot_graph(ax,x,y)
        path = sol[np.argmin(F_eval)]
        for line in path:
            ax.plot([x[line[0]], x[line[1]]], [y[line[0]], y[line[1]]], c='red', ls='-', alpha=1.0, linewidth=5, zorder=1)
        plt.show()


## Main Method

In [ ]:
%matplotlib notebook
if __name__ == '__main__':
    k = 5       # number of ants
    rho = 0.5
    gen_max = 20
    
    filename = 'Graph_bigger.txt'
    
    ACO(filename, k, gen_max, rho, plotting=True)

## Additional optional exercises
1. Create different graphs and apply the algorithm. Also change the parameters and see how it changes solutions.
2. The scheme implemented here is an advanced one regarding solution construction as well as pheromone approximation. Implement the basic scheme presented in the lecture.
3. Add a new termination criterion that stops the algorithm when all ants follow the same path.
4. Write a separate Python script to generate new graphs.